## Match Filardo data to PubMed and extract author:gender data

In [1]:
from Bio import Entrez
import pandas as pd
import string
import time
import random
import csv
import difflib

In [2]:
# Uncomment to display full text in cells
pd.set_option('display.max_colwidth', -1)

In [3]:
# Code from https://marcobonzanini.com/2015/01/12/searching-pubmed-with-python/
def search(query):
    Entrez.email = 'lucia.santamaria@ymail.com'
    handle = Entrez.esearch(db='pubmed', 
                            sort='relevance', 
                            retmax='20',
                            retmode='xml', 
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'lucia.santamaria@ymail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

In [4]:
df = pd.read_excel('Filardo_testdata.xlsx')
df.head()

,Article_ID,Full_Title,Link_to_Article,Journal,Year,Month,First Author Gender
0,Aaby et al. (2010),Non-specific effects of standard measles vaccine at 4.5 and 9 months of age on childhood mortality: randomised controlled trial,NaN,BMJ,2010,12 - Dec,Male
1,Aaron et al. (2007),Tiotropium in Combination with Placebo| Salmeterol| or Fluticasone–Salmeterol for Treatment of Chronic Obstructive Pulmonary Disease: A Randomized Trial,http://annals.org/article.aspx?articleid=734106,Annals of Internal Medicine,2007,04 - Apr,Male
2,Abbott et al. (2004),?-Blocker Use in Long-term Dialysis Patients Association With Hospitalized Heart Failure and Mortality,http://archinte.jamanetwork.com/article.aspx?articleid=217784,Archives of Internal Medicine,2004,12 - Dec,Male
3,Abdulla et al. (2008),Safety and Immunogenicity of RTS|S/AS02D Malaria Vaccine in Infants,http://www.nejm.org/doi/pdf/10.1056/NEJMoa0807773,NEJM,2008,12 - Dec,Male
4,Aboa-Eboule et al. (2007),Job Strain and Risk of Acute Recurrent Coronary Heart Disease Events,http://jama.jamanetwork.com/article.aspx?articleid=209121,JAMA,2007,10 - Oct,Female


In [5]:
def build_pubmed_query(row):
    # Extract first author, year
    # Remove final year and take everything before et al.
    author = ' '.join(row['Article_ID'].split()[:-1]).split('et al')[0].strip()
    collaboration_words = ['group', 'collaboration']
    if len(author.split()) > 3 or any(word in author.lower() for word in collaboration_words):
        # If author name has more than 3 words or it contains the coll words it's likely a collaboration - skip
        author = ''
    year = row['Year']
    # Process title -> remove punctuation and short words
    title = row['Full_Title'].translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    shorttitle = ' '.join([t for t in title.split() if len(t)>=5])
    return ' '.join([author, str(year), shorttitle]).strip()

In [6]:
df['pubmed_query'] = df.apply(lambda r: build_pubmed_query(r), axis=1)
df.head()

,Article_ID,Full_Title,Link_to_Article,Journal,Year,Month,First Author Gender,pubmed_query
0,Aaby et al. (2010),Non-specific effects of standard measles vaccine at 4.5 and 9 months of age on childhood mortality: randomised controlled trial,NaN,BMJ,2010,12 - Dec,Male,Aaby 2010 specific effects standard measles vaccine months childhood mortality randomised controlled trial
1,Aaron et al. (2007),Tiotropium in Combination with Placebo| Salmeterol| or Fluticasone–Salmeterol for Treatment of Chronic Obstructive Pulmonary Disease: A Randomized Trial,http://annals.org/article.aspx?articleid=734106,Annals of Internal Medicine,2007,04 - Apr,Male,Aaron 2007 Tiotropium Combination Placebo Salmeterol Fluticasone–Salmeterol Treatment Chronic Obstructive Pulmonary Disease Randomized Trial
2,Abbott et al. (2004),?-Blocker Use in Long-term Dialysis Patients Association With Hospitalized Heart Failure and Mortality,http://archinte.jamanetwork.com/article.aspx?articleid=217784,Archives of Internal Medicine,2004,12 - Dec,Male,Abbott 2004 Blocker Dialysis Patients Association Hospitalized Heart Failure Mortality
3,Abdulla et al. (2008),Safety and Immunogenicity of RTS|S/AS02D Malaria Vaccine in Infants,http://www.nejm.org/doi/pdf/10.1056/NEJMoa0807773,NEJM,2008,12 - Dec,Male,Abdulla 2008 Safety Immunogenicity AS02D Malaria Vaccine Infants
4,Aboa-Eboule et al. (2007),Job Strain and Risk of Acute Recurrent Coronary Heart Disease Events,http://jama.jamanetwork.com/article.aspx?articleid=209121,JAMA,2007,10 - Oct,Female,Aboa-Eboule 2007 Strain Acute Recurrent Coronary Heart Disease Events


In [7]:
def query_pubmed(query, title):
    results = search(query)
    if results['Count'] == '1':
        # If search returned a unique item the matching is straightforward
        paper = fetch_details(results['IdList'])
        pubmed_id = int(results['IdList'][0])
        author = paper['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList'][0]
    elif int(results['Count']) > 1 and int(results['Count']) <= 10:
        # If multiple papers (but less than 10) are returned, we compare titles and find the closest
        pubmed_ids = results['IdList']
        papers = fetch_details(pubmed_ids)
        titles = [p['MedlineCitation']['Article']['ArticleTitle'] for p in papers['PubmedArticle']]
        authors = [p['MedlineCitation']['Article']['AuthorList'] for p in papers['PubmedArticle']]
        to_match_title = title
        # Compare each of the titles with the original one
        scores = [(pid, author, difflib.SequenceMatcher(None, to_match_title.lower(), tit.lower()).ratio()) \
                  for (pid, author, tit) in zip(pubmed_ids, authors, titles)]
        # The matched article is that with the highest matching score
        matched = max(scores, key=lambda x:x[2])
        pubmed_id = int(matched[0])
        author = matched[1][0]
    else:
        return 0, None
    return pubmed_id, author

In [8]:
df.pubmed_query[0]

'Aaby 2010 specific effects standard measles vaccine months childhood mortality randomised controlled trial'

In [9]:
query_pubmed(df.pubmed_query[0], df.Full_Title[0])

(21118875,
 DictElement({'Identifier': [], 'AffiliationInfo': [{'Identifier': [], 'Affiliation': 'Bandim Health Project, Indepth Network, Bissau, Guinea-Bissau. p.aaby@bandim.org'}], 'LastName': 'Aaby', 'ForeName': 'Peter', 'Initials': 'P'}, attributes={'ValidYN': 'Y'}))

### Do not run this unless new results needed - read from file (section below) instead. API has usage limitations

In [10]:
# Run PubMed queries sequentially
# Sleep to avoid hitting the API too often. 
# Guidelines: https://www.ncbi.nlm.nih.gov/books/NBK25497/#chapter2.Usage_Guidelines_and_Requiremen
# Store failures for further retrieval
failed = []
for i, row in df.iterrows():
    if i % 100 == 0:
        print('Queried {} entries'.format(i))
    # Sleep 0 or 1 sec each time randomly
    time.sleep(random.randint(0,1))
    try:
        pubmed_id, author = query_pubmed(row['pubmed_query'], row['Full_Title'])
        df.loc[i,'pubmed_id'], df.loc[i,'ForeName'], df.loc[i,'LastName'] = pubmed_id, author['ForeName'], author['LastName']
    except:
        failed.append(i)

Queried 0 entries
Queried 100 entries
Queried 200 entries
Queried 300 entries
Queried 400 entries
Queried 500 entries
Queried 600 entries
Queried 700 entries
Queried 800 entries
Queried 900 entries
Queried 1000 entries
Queried 1100 entries
Queried 1200 entries
Queried 1300 entries
Queried 1400 entries
Queried 1500 entries
Queried 1600 entries
Queried 1700 entries
Queried 1800 entries
Queried 1900 entries
Queried 2000 entries
Queried 2100 entries
Queried 2200 entries
Queried 2300 entries
Queried 2400 entries
Queried 2500 entries
Queried 2600 entries
Queried 2700 entries
Queried 2800 entries
Queried 2900 entries
Queried 3000 entries
Queried 3100 entries
Queried 3200 entries


In [15]:
df[df.index.isin(failed)].head()

,Article_ID,Full_Title,Link_to_Article,Journal,Year,Month,First Author Gender,pubmed_query,pubmed_id,ForeName,LastName
13,ACTIVE Writing Group (2006),Clopidogrel plus aspirin versus oral anticoagulation for atrial fibrillation in the Atrial fibrillation Clopidogrel Trial with Irbesartan for prevention of Vascular Events (ACTIVE W): a randomised controlled trial,http://www.thelancet.com/journals/lancet/article/PIIS0140-6736(06)68845-4/abstract,The Lancet,2006,06 - Jun,Unknown,2006 Clopidogrel aspirin versus anticoagulation atrial fibrillation Atrial fibrillation Clopidogrel Trial Irbesartan prevention Vascular Events ACTIVE randomised controlled trial,NaN,NaN,NaN
34,Albalak et al. (2007),Rachel Albalak| PhD; Richard J. O’Brien| MD; J. Steve Kammerer| MBA; Sean M. O’Brien| PhD; Suzanne M. Marks| MPH| MA; Kenneth G. Castro| MD; Marisa Moore| MD,http://archinte.jamanetwork.com/article.aspx?articleid=770021,Archives of Internal Medicine,2007,12 - Dec,Female,Albalak 2007 Rachel Albalak Richard O’Brien Steve Kammerer O’Brien Suzanne Marks Kenneth Castro Marisa Moore,NaN,NaN,NaN
91,Aronson et al. (1997),Use of Testosterone To Prevent Cyclophosphamide-Induced Azoospermia,http://annals.org/article.aspx?articleid=710324,Annals of Internal Medicine,1997,02 - Feb,Male,Aronson 1997 Testosterone Prevent Cyclophosphamide Induced Azoospermia,NaN,NaN,NaN
94,Arteriovenous Malformation Study Group (1999),Arteriovenous Malformations of the Brain in Adults,http://www.nejm.org/doi/full/10.1056/NEJM199906103402307,NEJM,1999,06 - Jun,Unknown,1999 Arteriovenous Malformations Brain Adults,NaN,NaN,NaN
103,Atrial Fibrillation Investigators (1998),Echocardiographic Predictors of Stroke in Patients With Atrial Fibrillation: A Prospective Study of 1066 Patients From 3 Clinical Trials,http://archinte.jamanetwork.com/article.aspx?articleid=207367,Archives of Internal Medicine,1998,06 - Jun,Male,Atrial Fibrillation Investigators 1998 Echocardiographic Predictors Stroke Patients Atrial Fibrillation Prospective Study Patients Clinical Trials,NaN,NaN,NaN


In [12]:
len(df[df.pubmed_id.isnull()])

135

In [13]:
df.to_csv('Filardo_testdata_matched.csv', 
          quoting=csv.QUOTE_ALL,
          index=False)

### Start here if the export datafile is present

In [15]:
df = pd.read_csv('Filardo_testdata_matched.csv')

In [16]:
# Remove entries that weren't matched to pubmed
print('Entries in Filardo et al: {}'.format(len(df)))
print('Entries not matched to PubMed: {}'.format(len(df[df.pubmed_id.isnull()])))
df = df[~df.pubmed_id.isnull()]
df.reset_index(inplace=True)
df.head()

Entries in Filardo et al: 3204
Entries not matched to PubMed: 135


,index,Article_ID,Full_Title,Link_to_Article,Journal,Year,Month,First Author Gender,pubmed_query,pubmed_id,ForeName,LastName
0,0,Aaby et al. (2010),Non-specific effects of standard measles vacci...,NaN,BMJ,2010,12 - Dec,Male,Aaby 2010 specific effects standard measles va...,21118875.0,Peter,Aaby
1,1,Aaron et al. (2007),Tiotropium in Combination with Placebo| Salmet...,http://annals.org/article.aspx?articleid=734106,Annals of Internal Medicine,2007,04 - Apr,Male,Aaron 2007 Tiotropium Combination Placebo Salm...,17310045.0,Shawn D,Aaron
2,2,Abbott et al. (2004),?-Blocker Use in Long-term Dialysis Patients A...,http://archinte.jamanetwork.com/article.aspx?a...,Archives of Internal Medicine,2004,12 - Dec,Male,Abbott 2004 Blocker Dialysis Patients Associat...,15596637.0,Kevin C,Abbott
3,3,Abdulla et al. (2008),Safety and Immunogenicity of RTS|S/AS02D Malar...,http://www.nejm.org/doi/pdf/10.1056/NEJMoa0807773,NEJM,2008,12 - Dec,Male,Abdulla 2008 Safety Immunogenicity AS02D Malar...,19064623.0,Salim,Abdulla
4,4,Aboa-Eboule et al. (2007),Job Strain and Risk of Acute Recurrent Coronar...,http://jama.jamanetwork.com/article.aspx?artic...,JAMA,2007,10 - Oct,Female,Aboa-Eboule 2007 Strain Acute Recurrent Corona...,17925517.0,Corine,Aboa-Eboulé


In [17]:
def parse_forename(forename):
    name = str(forename).lower()
    if len(name.split()) > 1:
        first_name, middle_name = name.split()[:2]
    else:
        first_name, middle_name = name, ''
    return first_name, middle_name

In [18]:
gender_keys = {'Unknown': 'u', 
               'Male': 'm',
               'Female': 'f'}

In [25]:
test_data = pd.DataFrame()
test_data['first_name'], test_data['middle_name'] = zip(*df['ForeName'].map(parse_forename))
test_data['last_name'] = df['LastName'].apply(lambda n: str(n).lower())
test_data['gender'] = df['First Author Gender'].apply(lambda n: gender_keys[n])
test_data['full_name'] = test_data.apply(lambda x: ' '.join([x.first_name, x.middle_name, x.last_name]), axis=1)

In [26]:
test_data.head()

,first_name,middle_name,last_name,gender,full_name
0,peter,,aaby,m,peter aaby
1,shawn,d,aaron,m,shawn d aaron
2,kevin,c,abbott,m,kevin c abbott
3,salim,,abdulla,m,salim abdulla
4,corine,,aboa-eboulé,f,corine aboa-eboulé


In [27]:
test_data.to_csv('test_data_filardo.csv',
                 columns=["first_name", "middle_name", "last_name", "full_name", "gender"],
                 quoting=csv.QUOTE_ALL, index=False)